In [12]:
from googleapiclient.discovery import build

import requests

from IPython.core.display import display, HTML
from PIL import Image
import cStringIO
import base64
import pandas as pd

class MovieSearch:
    
    def __init__(self, my_api_key):
        # my cse, public url: https://cse.google.com/cse?cx=010816772880144122806:b374ikzbnui
        self.my_cse_id = "010816772880144122806:b374ikzbnui"
        self.my_api_key = my_api_key
        
        # Declare vars for data, not necessary for python but makes code clearer
        self.img = None
        self.rating = None
        self.IMDb_link = None
        self.wiki_link = None
        self.you_link = None
    
    def search_movie(self, movie_title):
        # 5 is enough for what we want, can set to 10 for more garunteed results, but slower
        search_res = self.google_search(movie_title, self.my_api_key, self.my_cse_id, num = 5)
        self.parse_result(search_res) # set up all vars
        self.print_table()
        
    # Display the search result in a table form and hyperlinks
    def print_table(self):
        
        with_rating = 'IMDb: {}/10'.format(self.rating)
        
        # pandas automatically adjust the size of column to fit all, last element goes into second line with max len: 49
        data = [['<img src="data:image/png;base64,{0:s}">', with_rating, self.IMDb_link, self.wiki_link, self.you_link]]
        col = ['Movie Poster', 'Rating', 'IMDb - With Reviews', 'Wikipedia - Info', 'YouTube - Trailer']
        df = pd.DataFrame(data, columns = col)

        imgStr = self.convert_image(self.img)
        # 
        html_all = df.to_html(escape=False).format(imgStr)
        display(HTML(html_all))
        
        print 'url as hyperlinks --'
        print 'Wikipeida: ',self.wiki_link
        print 'IMDb: ',self.IMDb_link
        print 'YouTube: ',self.you_link

    # Parse and store values for data 
    def parse_result(self, results):
        
        # To only pick the first occurence site and avoid overriding existing data 
        wiki_found = False
        youtube_found = False
        imdb_found = False
        
        for res in results:
            link = res['formattedUrl']
            title = res['title']
            site = title[title.rfind('-') + 2 : len(title)] #site name extracted
            
            if(site == 'IMDb' and (not imdb_found)):
                imdb_found = True
                self.rating = res['pagemap']['aggregaterating'][0]['ratingvalue']
                self.IMDb_link = link

            elif(site == 'Wikipedia' and  (not wiki_found)):
                wiki_found = True
                self.img = res['pagemap']['cse_image'][0]['src']
                self.wiki_link = link
                
            elif(site == 'YouTube' and  (not youtube_found)):
                youtube_found = True
                self.you_link = link
                
            elif(title.find('Trailer') != -1 and (not youtube_found)):
                youtube_found = True
                self.you_link = link
                
    # The main search process using google's api client and custom search engine 
    def google_search(self, search_term, api_key, cse_id, **kwargs):
        service = build("customsearch", "v1", developerKey=api_key)
        res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
        return res['items']
    
    # Convert an image in url form to @a base 64 in order to embedd in html
    def convert_image(self, img_url):
    
        # save image as url to local directory as .png
        img_data = requests.get(img_url).content
        with open('movie_image.png', 'wb') as handler:
            handler.write(img_data)

        # @to be written
        img = Image.open('<The path to current directory(saved image)>') #Don't add bracket
        img_buffer = cStringIO.StringIO()
        img.save(img_buffer, format="png")
        converted_img = base64.b64encode(img_buffer.getvalue())
        return converted_img 
    

In [ ]:
# Sample Code: 

google_api_key = "<Your Google API Key>" # Don't include the brackets 

# Movie title should be exact for best result; for movies that have series, make sure to include the full name.
# Google API has a limit of 100 searches/day (free); see its prices for more (link in readme)
movie_title = "<Title of the movie>"

test = MovieSearch(google_api_key)
test.search_movie(movie_title)